In [ ]:
import requests
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from spectools import lpsd
import scipy.signal.windows as win
from scipy.io import loadmat
import spectools.flattop as ft
from functools import partial

import multiprocessing as mp
ctx = mp.get_context('fork')
pool = ctx.Pool()

In [ ]:
plt.rcParams.update({
    # Figure and axes properties
    'figure.figsize': (5, 3),            # Figure size in inches
    'figure.dpi': 150,                   # Dots per inch for the figure (controls resolution)
    'figure.facecolor': 'white',         # Background color of the figure
    'figure.edgecolor': 'white',         # Edge color of the figure
    'axes.facecolor': 'white',           # Background color of the axes
    'axes.edgecolor': 'black',           # Edge color of the axes
    'axes.linewidth': 1.0,               # Line width of the axes' frame
    'axes.grid': True,                   # Whether to show grid lines by default
    'grid.color': 'gray',                # Grid line color
    'grid.linestyle': '--',              # Grid line style
    'grid.linewidth': 0.7,               # Grid line width

    # Font and text properties
    'font.size': 8,                      # Default font size
    'font.family': 'sans-serif',         # Font family
    'font.sans-serif': ['Arial'],        # Specify sans-serif font
    'axes.labelsize': 8,                 # Font size for axes labels
    'axes.titlesize': 9,                 # Font size for the plot title
    'axes.labelcolor': 'black',          # Color of the axes labels
    'axes.titleweight': 'regular',       # Weight of the title font
    'legend.fontsize': 8,                # Font size of the legend
    'legend.frameon': True,              # Whether to draw a frame around the legend

    # Tick properties
    'xtick.labelsize': 8,                # Font size for x-axis tick labels
    'ytick.labelsize': 8,                # Font size for y-axis tick labels
    'xtick.color': 'black',              # Color of x-axis ticks
    'ytick.color': 'black',              # Color of y-axis ticks
    'xtick.direction': 'out',            # Direction of x-axis ticks ('in' or 'out')
    'ytick.direction': 'out',            # Direction of y-axis ticks

    # Lines properties
    'lines.linewidth': 1.5,              # Default line width
    'lines.color': 'black',              # Default line color
    'lines.linestyle': '-',              # Default line style ('-', '--', '-.', ':')
    'lines.marker': '',                  # Default marker for points
    'lines.markersize': 5,               # Marker size

    # Legend properties
    'legend.loc': 'best',                # Legend location ('best', 'upper right', etc.)
    'legend.framealpha': 1.0,            # Transparency of the legend frame
    'legend.edgecolor': 'black',         # Edge color of the legend box

    # Save figure properties
    'savefig.dpi': 300,                  # Resolution when saving figures
    'savefig.format': 'pdf',             # Default format for saving figures
    'savefig.bbox': 'tight',             # Bounding box when saving figures
    'savefig.facecolor': 'white',        # Background color for saved figures

    # Color properties
    'axes.prop_cycle': plt.cycler('color', ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd']),
    # Define a color cycle for different plot lines
})

# Crayons Theme:
plt.rcParams['axes.prop_cycle'] = plt.cycler('color', [
    '#DC143C', '#FFD700', '#00BFFF', '#32CD32', '#FF69B4',
    '#FF4500', '#1E90FF', '#8A2BE2', '#FFA07A', '#8B0000',
    '#FF8C00', '#00FA9A', '#7B68EE', '#20B2AA', 
    '#FF6347', '#4682B4'
])

In [ ]:
# The following link should lead to a .mat datafile
# Contact the author at spectools@pm.me if the link is broken
url = 'https://www.dropbox.com/s/znmk8tfm0gv8a7b/GFO_Week52_2022.mat?dl=1'

# Download the file
response = requests.get(url)
with open('data.mat', 'wb') as f:
    f.write(response.content)

# Load the .mat file into a NumPy array
data = loadmat('data.mat')
x = np.array(data['RawPhase'][:,0])
print(x)

fs=9.664

In [ ]:
figsize=(8,3)
dpi=150
fontsize=8
linewidth=1.5
xlabel="Sample"
ylabel="Signal"
title="GRACE Follow On LRI range in meters - Calendar week 52, 2022"

fig, ax = plt.subplots(figsize=figsize, dpi=dpi)
ax.plot(x, linewidth=linewidth, label="test", color="gray")
ax.set_xlabel(xlabel, fontsize=fontsize)
ax.set_ylabel(ylabel, fontsize=fontsize)
ax.set_title(title, fontsize=fontsize)
ax.tick_params(labelsize=fontsize)
ax.grid()
fig.tight_layout()
fig.align_ylabels()
plt.show()

In [ ]:
# List of window functions available in scipy.signal.windows
windows_sc = {name: getattr(win, name) for name in dir(win) if callable(getattr(win, name))}
windows_sc

In [ ]:
# List of window functions available in spectools.flattop
windows_ft = {name: getattr(ft, name) for name in dir(ft) if callable(getattr(ft, name))}
windows_ft

In [ ]:
# List of window functions available in scipy.signal.windows
# windows = {name: getattr(win, name) for name in dir(win) if callable(getattr(win, name))}
# windows

windows = {
    'boxcar': win.boxcar,
    'triang': win.triang,
    # 'blackman': win.blackman,
    # 'hamming': win.hamming,
    'hann': win.hann,
    # 'bartlett': win.bartlett,
    # 'flattop': win.flattop,
    # 'parzen': win.parzen,
    # 'bohman': win.bohman,
    # 'blackmanharris': win.blackmanharris,
    'nuttall': partial(win.nuttall, sym=False),
    # 'barthann': win.barthann,
    'kaiser': win.kaiser,
    # 'tukey': win.tukey
    # "SFT3F": ft.SFT3F,
    # "SFT3M": ft.SFT3M,
    # "FTNI": ft.FTNI,
    # "SFT4F": ft.SFT4F,
    # "SFT5F": ft.SFT5F,
    # "SFT4M": ft.SFT4M,
    # "FTHP": ft.FTHP,
    "HFT70": ft.HFT70,
    # "FTSRS": ft.FTSR,
    # "SFT5M": ft.SFT5M,
    # "HFT90D": ft.HFT90D,
    # "HFT95": ft.HFT95,
    # "HFT116D": ft.HFT116D,
    # "HFT144D": ft.HFT144D,
    # "HFT169D": ft.HFT169D,
    # "HFT196D": ft.HFT196D,
    # "HFT223D": ft.HFT223D,
    # "HFT248D": ft.HFT248D
    }
windows

In [ ]:
res = {} # Dictionary to store results

In [ ]:
olap = 0.75
Jdes = 1000
Kdes = 100
force_Jdes = True
scheduler='lpsd'

for win_str, win_fun in tqdm(windows.items()):
    res[win_str] = lpsd.ltf(x, 
                            fs=fs, 
                            win=win_fun, 
                            olap=olap, 
                            Jdes=Jdes, 
                            Kdes=Kdes, 
                            adjust_Jdes=force_Jdes, 
                            scheduler=scheduler, 
                            pool=pool)

In [ ]:
fig, ax = plt.subplots()

for win_str, psd in res.items():
    ax.loglog(psd.f, psd.asd, lw=linewidth, label=win_str)
ax.set_xlim([psd.f[0],psd.f[-1]])
ax.set_xlabel("Fourier frequency (Hz)")
ax.set_ylabel(r"ASD $\rm (A/Hz^{1/2})$")
ax.set_title("Comparison of different window functions in a high dynamic range signal")
ax.legend()
plt.show()